In [1]:
using DirectTrajectoryOptimization
using LinearAlgebra
using CairoMakie

Setting up quantum optimal control problem for a single qubit with Hamiltonian given by

$$
H(a(t)) = H_{\text{drift}} + a(t) H_{\text{drive}}
$$ 

where $H_{\text{drift}} = \sigma_z$ and $H_{\text{drive}} = \sigma_x$.

We will also define the qubit basis kets (unit vectors in $\mathbb{C}^2$) $\ket{0}$ and $\ket{1}$.

In [2]:
X = [
    0 1;
    1 0
]

Y = [
    0 -im;
    im 0
]

Z = [
    1 0;
    0 -1
]

f = 0.5

H_drift = f * Z

H_drive = X

ψ0 = [1, 0]
ψ1 = [0, 1];


below will define the specific problem we will solve:

In [3]:
gate = X
ψi = ψ0
ψf = X * ψi

2-element Vector{Int64}:
 0
 1

Now we define the time-dependent Schroedinger equation dynamics:

$$
{d \over dt} \ket{\psi} = -i H \ket{\psi}
$$

Here we use the isomorphism $\mathbb{C}^2 \cong \mathbb{R}^4$ 

$$
\ket{\psi} \equiv \begin{pmatrix} \psi_0 \\ \psi_1 \end{pmatrix} \cong \begin{pmatrix} \psi_0^{\textrm{Re}} \\ \psi_0^{\textrm{Im}} \\ \psi_1^{\textrm{Re}} \\ \psi_1^{\textrm{Im}} \\ \end{pmatrix} \equiv \tilde \psi
$$

and for a matrix $H \in \mathbb{C}^{2 \times 2}$

$$
\tilde H = \begin{pmatrix} H^{\textrm{Re}} & -H^{\textrm{Im}} \\ H^{\textrm{Im}} & H^{\textrm{Re}} \end{pmatrix} 
= \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \otimes H^{\textrm{Re}} + \begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix} \otimes H^{\textrm{Im}}
$$

which implies that for a matrix in the form $-i H$
$$
G(H) \equiv \widetilde{\left( -i H \right)} = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \otimes H^{\textrm{Im}} - \begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix} \otimes H^{\textrm{Re}}
$$

In our case $G$ (named for the generator of time translation) can be written as

$$
G \equiv G(H(a(t))) = G(H_{\textrm{drift}}) + a(t)G(H_{\textrm{drive}})
$$

Now we can write the isomorphic dynamics equation:

$$
\begin{equation}
{d \over dt} \tilde \psi = G \ \tilde \psi
\end{equation}
$$

Let's now define functions to compute these isomorphisms, as well as get the real and imaginary parts of the Hamiltonian:

In [4]:
ket_to_iso(ψ) = [real(ψ); imag(ψ)]
iso_to_ket(ψ̃) = [ψ̃[1] + im * ψ̃[2], ψ̃[3] + im * ψ̃[4]]

Id2 = I(2)
Im2 = [
    0 -1; 
    1  0
]

⊗(A, B) = kron(A, B)

G(H) = Id2 ⊗ imag(H) - Im2 ⊗ real(H) 

G_drift = G(H_drift)
G_drive = G(H_drive);

and define the dynamics:

In [5]:
schroedinger(x, u, w) = (G_drift + u[1] * G_drive) * x

schroedinger (generic function with 1 method)

In [6]:
xi = ket_to_iso(ψi)
xf = ket_to_iso(ψf);

In [7]:
function cost(x, xf)
    ψ = iso_to_ket(x)
    ψf = iso_to_ket(xf)
    return min(abs(1 - ψ'ψf), abs(1 + ψ'ψf))
end

costi(x) = cost(x, xi)
costf(x) = cost(x, xf)

costf (generic function with 1 method)

In [8]:
function midpoint_implicit(y, x, u, w)
    h = 0.01
    return y - (x + h * schroedinger(0.5 * (x + y), u, w))
end

midpoint_implicit (generic function with 1 method)

In [9]:
T = 11
num_state = 4
num_action = 1
eval_hess=true;

In [10]:
dt = Dynamics(
    midpoint_implicit,
    num_state,
    num_state,
    num_action,
    evaluate_hessian=eval_hess
)
dynamics = [dt for t = 1:T-1];

In [11]:
ot = (x, u, w) -> costf(x) .+ 0.1 * dot(u, u)
oT = (x, u, w) -> costf(x)
ct = Cost(ot, num_state, num_action; evaluate_hessian=eval_hess)
cT = Cost(oT, num_state, num_action; evaluate_hessian=eval_hess)
objective = [[ct for t = 1:T-1]; cT];

In [12]:
bnd1 = Bound(num_state, num_action)
bndt = Bound(num_state, num_action)
bndT = Bound(num_state, 0)
bounds = [bnd1, [bndt for t = 2:T-1]..., bndT];

In [13]:
con1 = Constraint((x, u, w) -> [costi(x)], num_state, num_action, evaluate_hessian=eval_hess)
conT = Constraint((x, u, w) -> [costf(x)], num_state, num_action, evaluate_hessian=eval_hess) 
constraints = [con1; [Constraint() for t = 2:T-1]; conT];

In [14]:
solver = Solver(dynamics, objective, constraints, bounds);

In [15]:
solve!(solver)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      284
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0



LoadError: DimensionMismatch("array could not be broadcast to match destination")